In [3]:
import json
import numpy as np
import pandas as pd 

In [4]:
file_dir = 'wikipedia-movies.json'

In [5]:
with open(file_dir, mode='r') as file:
    wiki_movies_raw = json.load(file)

In [6]:
kaggle_metadata = pd.read_csv('archive/movies_metadata.csv', low_memory=False)
ratings = pd.read_csv('archive/ratings.csv')

In [7]:
wiki_movies_df = pd.DataFrame(wiki_movies_raw)

In [8]:
wiki_movies = [
    movie for movie in wiki_movies_raw
    if ('Director' in movie or 'Directed by' in movie)
    and 'imdb_link' in movie
    and 'No. of episodes' not in movie
] 

In [19]:
def clean_movie(movie):
    '''
    This function takes in a movie from our 'wikipedia-movies.json'. 
    If one or more alternate titles exist, those are added to a new 'alt_title' key, which will manifest as a column in our subsequently created wiki_movies_df DataFrame. 
    Additionally, it will take remove redundant columns, adding the value to the desired new column name. 
    It returns the cleaned-up movie dictionary. 
    '''
    movie = dict(movie)
    
    alt_titles = {}
    alt_title_keys =[    
        'Also known as','Arabic','Cantonese','Chinese','French',
        'Hangul','Hebrew','Hepburn','Japanese','Literally',
        'Mandarin','McCune–Reischauer','Original title','Polish',
        'Revised Romanization','Romanized','Russian',
        'Simplified','Traditional','Yiddish'
    ]
    for key in alt_title_keys:
        if key in movie:
            alt_titles[key] = movie[key]
            movie.pop(key)
            
    if len(alt_titles) > 0:
        movie['alt_titles'] = alt_titles
        
    def change_column_name(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    
    # a list of column names to use with change_column_name(); each tuple is the old name and the desired new name
    column_names_to_change = [
        ('Adaptation by', 'Writer(s)'), ('Country of origin', 'Country'), ('Directed by', 'Director'), 
        ('Distributed by', 'Distributor'), ('Edited by', 'Editor(s)'), ('Length', 'Running time'), 
        ('Original release', 'Release date'), ('Music by', 'Composer(s)'), ('Produced by', 'Producer(s)'), 
        ('Producer', 'Producer(s)'), ('Productioncompanies ', 'Production company(s)'), ('Productioncompany ', 'Production company(s)'),
        ('Released', 'Release Date'), ('Release Date', 'Release date'), ('Screen story by', 'Writer(s)'), 
        ('Screenplay by', 'Writer(s)'), ('Story by', 'Writer(s)'), ('Theme music composer', 'Composer(s)'), 
        ('Written by', 'Writer(s)')
    ]
    
    for old_name, new_name in column_names_to_change:
        change_column_name(old_name, new_name)
        
    return movie

In [20]:
clean_movies = [clean_movie(movie) for movie in wiki_movies]

In [21]:
wiki_movies_df = pd.DataFrame(clean_movies)

In [22]:
sorted(wiki_movies_df.columns.tolist())

['Animation by',
 'Audio format',
 'Based on',
 'Box office',
 'Budget',
 'Cinematography',
 'Color process',
 'Composer(s)',
 'Country',
 'Created by',
 'Director',
 'Distributor',
 'Editor(s)',
 'Executive producer(s)',
 'Followed by',
 'Genre',
 'Label',
 'Language',
 'Narrated by',
 'Original language(s)',
 'Original network',
 'Picture format',
 'Preceded by',
 'Producer(s)',
 'Production company(s)',
 'Production location(s)',
 'Recorded',
 'Release date',
 'Running time',
 'Starring',
 'Suggested by',
 'Venue',
 'Voices of',
 'Writer(s)',
 'alt_titles',
 'imdb_link',
 'title',
 'url',
 'year']